In [3]:
#Importing the imdb dataset and essential libraries
from keras.datasets import imdb
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
#Fix a random seed
np.random.seed(13)

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "C:\Users\skitt\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\skitt\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\skitt\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\skitt\Anaconda3\lib\imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\skitt\Anaconda3\lib\imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: The specified module could not be found.


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/install_sou

In [5]:
##Preprocessing
#load text t
filename = "../resources/clean_dataset.txt"
raw_text = open(filename).read()

filename = "../resources/source_data.txt"
input_text = open(filename).read()

filename = "../resources/target_data.txt"
output_text = open(filename).read()


#Splitting text into a word per element array and punctuation also is its own element
#python does not distingiush between " and ' therfore do not worry about words that 
#contain ' and appear in "" 
split_raw_text = re.findall("[\w']+|[.,!?;:-]" , raw_text)	

# estimate vocab size
vocab = set(split_raw_text)
vocab_size = len(vocab)

tokenizer = Tokenizer(num_words=vocab_size)

#fit the tokenizer on the text 
tokenizer.fit_on_texts(raw_text)

# integer encode input and output
encoded_input = tokenizer.texts_to_matrix(input_text, mode='count')
encoded_output = tokenizer.texts_to_matrix(output_text, mode='count')

#Load only the top 20 words. 10 for testing, 10 for training.
word_pool_size = 20
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=word_pool_size)
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

#Creating the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(word_pool_size, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
#Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

FileNotFoundError: [Errno 2] No such file or directory: 'dataset_small_test_file.txt'

In [4]:
#Load only the top 20 words. 10 for testing, 10 for training.
word_pool_size = 20
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=word_pool_size)
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

17465344/17464789 [==============================] - 2s 0us/step


In [6]:
#Creating the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(word_pool_size, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
#Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           640       
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 53,941
Trainable params: 53,941
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
25000/25000 [==============================] - 114s 5ms/step - loss: 0.6890 - acc: 0.5336
Epoch 2/3
25000/25000 [==============================] - 115s 5ms/step - loss: 0.6687 - acc: 0.5906
Epoch 3/3
25000/25000 [==============================] - 118s 5ms/step - loss: 0.6674 - acc: 0.5934
Accuracy: 53.78%


This test provided by https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/
From their paper http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.bib